In [1]:
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from objetos_otimizacao_rede_neural import *
from lightning.pytorch.loggers import TensorBoardLogger
from optuna import create_study, Trial
import os

In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 0
NUM_EPOCAS = 2

In [3]:
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs/")
treinador = L.Trainer(logger=False,enable_checkpointing=True,max_epochs=NUM_EPOCAS)


def build_model(trial):
    num_dados_de_entrada = 5
    num_dados_de_saida = 1
    n_layers = trial.suggest_int("n_layers",2,5)
    neuronios_camadas =[]
    vieses = []
    for i in range(n_layers):
        neuronios =  trial.suggest_int(f"c{i}",2,5)
        bia = trial.suggest_categorical(f"bias{i}",[False, True])
        neuronios_camadas.append(neuronios)
        vieses.append(bia)
    
    minha_mlp = MLP(
        num_dados_de_entrada,list(neuronios_camadas), list(vieses), num_dados_de_saida
    )
    
    
    return minha_mlp

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
def objective(trial):

    k = []
    for i in range(int(1/TAMANHO_TESTE)):
        print(i)
        dm = DataModule(TAMANHO_TESTE, SEMENTE_ALEATORIA,i)
        minha_mlp = build_model(trial)
        treinador.fit(minha_mlp, dm)

        minha_mlp.eval()
        dm.setup("test")

        with torch.no_grad():
            X_true = dm.X_teste

            y_true = dm.y_teste
            y_true = dm.y_scaler.inverse_transform(y_true)

            y_pred = minha_mlp(X_true)
            y_pred = dm.y_scaler.inverse_transform(y_pred)

            RMSE = root_mean_squared_error(y_true, y_pred)
        
        k.append(RMSE)

    rmse_medio = (sum(np.array(k)**2)/int(1/TAMANHO_TESTE))**.5
    return rmse_medio



In [5]:
%%capture
study = create_study(direction='minimize')
parametros_totais = []

for _ in range(10):

    study.optimize(objective, n_trials=1)
    study.trials_dataframe().to_excel('triagem.xlsx')


[I 2024-05-02 11:09:34,780] A new study created in memory with name: no-name-ab58460d-e23b-488f-be58-4ed5c4cbddbb

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 45    
---------------------------------------
45        Trainable params
0         Non-trainable params
45        Total params
0.000     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=2` reached.

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 45    
---------------------------------------
45        Trainable params
0         Non-trainable params
45        Total params
0.000     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=2` reached.

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 45    
---------------------------------------
45        Trainable params
0         Non-trainable params
45        Total params
0.00